In [25]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [26]:
# Set the path to the dataset
data_dir = 'C:/Users/Nikita/Desktop/extra/dataset/images'

In [27]:
# Set the batch size and input image size for the model
batch_size = 32
input_shape = (224, 224, 3)

In [28]:
# Set the number of classes in the dataset
num_classes = 4

In [29]:
# Set the number of epochs to train the model for
num_epochs = 50

In [30]:
# Set up data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [31]:
# Set up data augmentation for validation data
val_datagen = ImageDataGenerator(rescale=1./255)

In [32]:
# Create data generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, ''),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, ''),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 944 images belonging to 4 classes.
Found 944 images belonging to 4 classes.


In [33]:
# Load the MobileNetV2 model without the top layers
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

In [34]:
# Add a new top layer for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [35]:
# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)


In [36]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [37]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
# Train the model
model.fit(train_generator,
          epochs=num_epochs,
          validation_data=val_generator,
          callbacks=[EarlyStopping(patience=3, restore_best_weights=True)])

Epoch 1/50
30/30 [==============================] - 56s 2s/step - loss: 1.6562 - accuracy: 0.2807 - val_loss: 1.2703 - val_accuracy: 0.4036
Epoch 2/50
30/30 [==============================] - 54s 2s/step - loss: 1.3126 - accuracy: 0.3898 - val_loss: 1.2029 - val_accuracy: 0.4364
Epoch 3/50
30/30 [==============================] - 53s 2s/step - loss: 1.2722 - accuracy: 0.4195 - val_loss: 1.1375 - val_accuracy: 0.5519
Epoch 4/50
30/30 [==============================] - 53s 2s/step - loss: 1.2079 - accuracy: 0.4619 - val_loss: 1.0751 - val_accuracy: 0.5784
Epoch 5/50
30/30 [==============================] - 52s 2s/step - loss: 1.1743 - accuracy: 0.4788 - val_loss: 1.0094 - val_accuracy: 0.6133
Epoch 6/50
30/30 [==============================] - 54s 2s/step - loss: 1.1388 - accuracy: 0.4968 - val_loss: 1.0154 - val_accuracy: 0.5922
Epoch 7/50
30/30 [==============================] - 54s 2s/step - loss: 1.1359 - accuracy: 0.4873 - val_loss: 0.9554 - val_accuracy: 0.6737
Epoch 8/50
30/30 [==

In [8]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [9]:
app = Flask(__name__)

In [10]:
# Define the emotion labels
emotion_labels = {0: 'angry', 1: 'happy', 2: 'sad' , 3:'relaxed'}

In [11]:
@app.route('/predict', methods=['POST'])
def predict():
    # Get the image from the request
    img = request.files['image'].read()

    # Preprocess the image
    img = image.img_to_array(image.load_img(io.BytesIO(img), target_size=(224, 224))) / 255.
    img = np.expand_dims(img, axis=0)

    # Make predictions
    pred = model.predict(img)
    pred_label = emotion_labels[np.argmax(pred)]

    # Return the prediction as JSON
    return jsonify({'emotion': pred_label})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1